# Exploratory Data Analysis
Familiarize with the dataset, we'll try to target question 3.

## Environment Setup
Load modules, data.

In [2]:
# modules
## external
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## stdlib
from pathlib import Path

## utils

In [6]:
# load in datasets
home_dir = Path().cwd().parent                                                  # github repo home dir

## financial data
economic_df = pd.read_csv(
    home_dir / "datasets" / "acs_5yr_est_selected_economic_characteristics_2010-2022.csv",
    engine="c", low_memory=False
)
commodities_df = pd.read_csv(
    home_dir / "datasets" / "all_commodities.csv",
    engine="c", low_memory=False
)
stocks_df = pd.read_csv(
    home_dir / "datasets" / "all_stock_and_etfs.csv",
    engine="c", low_memory=False
)
stock_descr_df = pd.read_csv(
    home_dir / "datasets" / "stock_descriptions.csv",
    engine="c", low_memory=False
)

## meat data
meat_storage_df = pd.read_csv(
    home_dir / "datasets" / "Meat_Stats_Cold_Storage.csv", engine="c",
    low_memory=False
)
meat_production_df = pd.read_csv(
    home_dir / "datasets" / "Meat_Stats_Meat_Production.csv", engine="c",
    low_memory=False
)
meat_slaughter_weights_df = pd.read_csv(
    home_dir / "datasets" / "Meat_Stats_Slaughter_Counts.csv", engine="c",
    low_memory=False
)
meat_slaughter_counts_df = pd.read_csv(
    home_dir / "datasets" / "Meat_Stats_Slaughter_Weights.csv", engine="c",
    low_memory=False
)
nutrition_df = pd.read_csv(
    home_dir / "datasets" / "Nutrition_Physical_Activity_and_Obesity_Data.csv", 
    engine="c", low_memory=False
)

## Exploratory Analysis
Distributions, plots, etc.